In [1]:
import torch, sys
sys.path.insert(0, '../')
from my_utils import gpu_utils
import importlib, gc
from my_utils.alignment_features import *
import my_utils.alignment_features as afeatures
importlib.reload(afeatures)
import gnn_utils.graph_utils as gutils

/mounts/Users/student/ayyoob/anaconda3/envs/multalign_graph/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# !pip install torch-geometric
# !pip install tensorboardX

# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip

#  print(torch.version.cuda)
#  print(torch.__version__)    

dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [3]:

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.transforms as T

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt




In [4]:
from my_utils import align_utils as autils, utils
import argparse
from multiprocessing import Pool
import random

# set random seed
config_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc-ui-demo/config_pbc.ini"
utils.setup(config_file)

params = argparse.Namespace()


params.gold_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi-fin-grc-gold-alignments_train.txt"
pros, surs = autils.load_gold(params.gold_file)
all_verses = list(pros.keys())
params.gold_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi-fin-heb-gold-alignments_train.txt"
pros, surs = autils.load_gold(params.gold_file)
all_verses.extend(list(pros.keys()))
all_verses = list(set(all_verses))
print(len(all_verses))

params.editions_file =  "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi_lang_list.txt"
editions, langs = autils.load_simalign_editions(params.editions_file)
current_editions = [editions[lang] for lang in langs]

def get_pruned_verse_alignments(args):
    verse, current_editions = args
    
    verse_aligns_inter = autils.get_verse_alignments(verse)
    verse_aligns_gdfa = autils.get_verse_alignments(verse, gdfa=True)

    autils.prune_non_necessary_alignments(verse_aligns_inter, current_editions)
    autils.prune_non_necessary_alignments(verse_aligns_gdfa, current_editions)

    gc.collect()
    return verse_aligns_inter, verse_aligns_gdfa
    

verse_alignments_inter = {}
verse_alignments_gdfa = {}
args = []
for i,verse in enumerate(all_verses):
    args.append((verse, current_editions[:]))

#print('going to get alignments')
#with Pool(20) as p:
#    all_res = p.map(get_pruned_verse_alignments, args)

#for i,verse in enumerate(all_verses):
#    verse_aligns_inter, verse_aligns_gdfa = all_res[i]
    
    #verse_alignments_inter[verse] = verse_aligns_inter
    #verse_alignments_gdfa[verse] = verse_aligns_gdfa

#utils.LOG.info("done reading alignments")
#torch.save(verse_alignments_inter, "/mounts/work/ayyoob/models/gnn/pruned_alignments_train_inter.pickle")
#torch.save(verse_alignments_gdfa, "/mounts/work/ayyoob/models/gnn/pruned_alignments_train_gdfa.pickle")
#utils.LOG.info('done saving pruned alignments')

print('reading inter verse alignments')
verse_alignments_inter = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_train_inter.pickle")
gc.collect()
print('done reading inter verse alignments')

for verse in all_verses[:]:
    if len(verse_alignments_inter[verse].keys()) < 10:
        all_verses.remove(verse)

24159
reading inter verse alignments
done reading inter verse alignments


In [5]:
#importlib.reload(afeatures)
class Encoder2(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Encoder2, self).__init__()
        self.conv1 = pyg_nn.GATConv(in_channels, 2*out_channels)
        self.conv2 = pyg_nn.GATConv(2 * out_channels , out_channels)

    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index, ))
        return self.conv2(x, edge_index)

class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels, features, n_head = 2, edge_feature_dim = 0,):
        super(Encoder, self).__init__()
        #self.lin = nn.Linear(in_channels, out_channels)
        self.conv1 = pyg_nn.GATConv(in_channels, 2*out_channels, heads= n_head)
        self.conv2 = pyg_nn.GATConv(2 * n_head *  out_channels , out_channels, heads= 1)
        #self.conv3 = pyg_nn.GATConv(2 * n_head *  out_channels , out_channels, heads= n_head)
        #self.f_embedding = nn.Linear(in_channels, in_channels)
        self.fin_lin = nn.Linear(out_channels, out_channels)
        

        self.feature_encoder = afeatures.FeatureEncoding(features, word_vectors)

    def forward(self, x, edge_index):
        x = self.feature_encoder(x, dev)
        x = F.elu(self.conv1(x, edge_index, ))
        #x = self.conv_gin(x, edge_index)
        x = F.elu(self.conv2(x, edge_index))
        return F.relu(self.fin_lin(x))#, self.conv3(x, edge_index)

In [6]:
class POSDecoder(nn.Module):
    def __init__(self, input_size, hidden_size, n_class, drop_out=0):
        super(POSDecoder, self).__init__()

        self.transfer = nn.Sequential(nn.Linear(input_size, hidden_size), nn.ReLU(), nn.Dropout(drop_out),
                        nn.Linear(hidden_size, n_class))

    def forward(self, z, index):
        h = z[index, :]

        res = self.transfer(h)

        return res

In [7]:
def clean_memory():
    gc.collect()
    with torch.no_grad():
        torch.cuda.empty_cache()

def train(epoch):
    global optimizer
    total_loss = 0
    model.train()

    for i,batch_ in enumerate(tqdm(data_loader)):
        for verse in batch_:
            if verse in masked_verses:
                continue
            batch = batch_[verse]
            optimizer.zero_grad()
            
            x = batch['x'].to(dev)
            edge_index = batch['edge_index'].to(dev)
            if torch.max(edge_index) >= x.shape[0]:
                print(torch.max(edge_index), x.shape)
                print(batch)
                break

            try:
                z = model.encode(x, edge_index)
            except Exception as e:
                global sag, khar, gav
                sag, khar, gav =  (i, batch_, verse)
                print(e)
                1/0

            nodes = batch['nodes'].to(dev)
            
            preds = model.decoder(z, nodes)
            
            labels = batch['labels'].to(dev)
            labels = torch.max(labels, dim=1).indices

            loss = criterion(labels, preds)

            loss.backward()
            optimizer.step()
            total_loss += loss.item() 
            
    
    writer.add_scalar("loss", total_loss, epoch)
    print(f"train loss: {total_loss}")
    print(f"cluster loss: {cluster_loss}")


In [8]:
def test(data_loader):
    model.eval()
    tot_correct = tot = 0
    with torch.no_grad():
        for i,batch_ in enumerate(tqdm(data_loader)):
            for verse in batch_:
                batch = batch_[verse]
                
                x = batch['x'].to(dev)
                edge_index = batch['edge_index'].to(dev)
                if torch.max(edge_index) >= x.shape[0]:
                    print(torch.max(edge_index), x.shape)
                    print(batch)
                    break

                try:
                    z = model.encode(x, edge_index)
                except Exception as e:
                    global sag, khar, gav
                    sag, khar, gav =  (i, batch_, verse)
                    print(e)
                    1/0

                nodes = batch['nodes'].to(dev)
                
                preds = model.decoder(z, nodes)
                preds = torch.max(preds, dim=1).indices
                
                labels = batch['labels'].to(dev)
                labels = torch.max(labels, dim=1).indices
                
                corrects = preds == labels
                tot += preds.shape[0]
                tot_correct += corrects.sum().float()


    return tot_correct/tot

In [10]:
import pickle

train_verses = all_verses[:]
test_verses = all_verses[:] 
editf1 = 'fin-x-bible-helfi'
editf2 = "heb-x-bible-helfi"


if 'jpn-x-bible-newworld' in  current_editions[:]:
     current_editions.remove('jpn-x-bible-newworld')
if 'grc-x-bible-unaccented' in  current_editions[:]:
     current_editions.remove('grc-x-bible-unaccented')

train_dataset = torch.load("/mounts/work/ayyoob/models/gnn/dataset_helfi_train_community_word.pickle", map_location=torch.device('cpu'))
#train_dataset, train_nodes_map = create_dataset(train_verses, verse_alignments_inter, small_editions)
features = train_dataset.features
train_nodes_map = train_dataset.nodes_map
#edge_index_intra_sent = train_dataset.edge_index_intra_sent
#test_edge_index_intra_sent = edge_index_intra_sent

# test_dataset, test_nodes_map = create_dataset(test_verses, verse_alignments_inter, small_editions)
test_dataset, test_nodes_map = train_dataset, train_nodes_map
test_verses = train_verses
print(train_dataset.x.shape)

gc.collect()


torch.Size([30723771, 10])


0

In [11]:
import codecs
import collections

postag_map = {"ADJ": 0, "ADP": 1, "ADV": 2, "AUX": 3, "CCONJ": 4, "DET": 5, "INTJ": 6, "NOUN": 7, "NUM": 8, "PART": 9, "PRON": 10, "PROPN": 11, "PUNCT": 12, "SCONJ": 13, "SYM": 14, "VERB": 15, "X": 16}

pos_lang_list = ["eng-x-bible-mixed", "deu-x-bible-newworld", "ces-x-bible-newworld", 
		"fra-x-bible-louissegond","hin-x-bible-newworld", "ita-x-bible-2009", 
		"prs-x-bible-goodnews", "ron-x-bible-2006", "spa-x-bible-newworld"]

def get_pos_tags(dataset):
	all_tags = {}
	for lang in pos_lang_list:
		if lang not in dataset.nodes_map:
			continue
		all_tags[lang] = {}
		with codecs.open(F"/mounts/work/mjalili/projects/gnn-align/data/pbc_pos_tags/{lang}.conllu", "r", "utf-8") as lang_pos:
			tag_sent = []
			sent_id = ""
			for sline in lang_pos:
				sline = sline.strip()
				if sline == "":
					if sent_id not in dataset.nodes_map[lang]:
						tag_sent = []
						sent_id = ""
						continue

					all_tags[lang][sent_id] = [p[3] for p in tag_sent]
					tag_sent = []
					sent_id = ""
				elif "# verse_id" in sline:
					sent_id = sline.split()[-1]
				elif sline[0] == "#":
					continue
				else:
					tag_sent.append(sline.split("\t"))

	pos_labels = torch.zeros((dataset.x.shape[0], len(postag_map)))
	pos_node_cover = collections.defaultdict(list)

	for lang in all_tags:
		for sent_id in all_tags[lang]:
			sent_tags = all_tags[lang][sent_id]
			for w_i in range(len(sent_tags)):
				if w_i not in dataset.nodes_map[lang][sent_id]:
					continue
				pos_labels[dataset.nodes_map[lang][sent_id][w_i], postag_map[sent_tags[w_i]]] = 1
				pos_node_cover[sent_id].append(dataset.nodes_map[lang][sent_id][w_i])

	return pos_labels, pos_node_cover
	#pos_pickle = {"pos_labels": pos_labels, "node_ids_train": pos_ids_train, "node_ids_dev": pos_ids_dev}
	#torch.save(pos_pickle, '/mounts/work/ayyoob/models/gnn/postag')


In [12]:
#from gensim.models import Word2Vec
#w2v_model = Word2Vec.load("/mounts/work/ayyoob/models/w2v/word2vec_helfi_langs_15e.model")

#print(w2v_model.wv.vectors.shape)

#word_vectors = torch.from_numpy(w2v_model.wv.vectors).float()

#print(word_vectors.shape)

In [13]:
blinker_test_dataset = torch.load("/mounts/work/ayyoob/models/gnn/dataset_blinker_full_community_word.pickle", map_location=torch.device('cpu'))
editf12 = "eng-x-bible-mixed"
editf22 = 'fra-x-bible-louissegond'

test_gold_eng_fra = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/eng_fra_pbc/eng-fra.gold"

pros_blinker, surs_blinker = autils.load_gold(test_gold_eng_fra)
blinker_verses = list(pros_blinker.keys())

#blinker_verse_alignments_inter = {}
#blinker_verse_alignments_gdfa = {}
#args = []
#for i,verse in enumerate(blinker_verses):
#    args.append((verse, current_editions))

#with Pool(20) as p:
#    all_res = p.map(get_pruned_verse_alignments, args)

#for i,verse in enumerate(blinker_verses):
#    verse_aligns_inter, verse_aligns_gdfa = all_res[i]
    
#    blinker_verse_alignments_inter[verse] = verse_aligns_inter
#    blinker_verse_alignments_gdfa[verse] = verse_aligns_gdfa

utils.LOG.info("done reading alignments")
#torch.save(blinker_verse_alignments_inter, "/mounts/work/ayyoob/models/gnn/pruned_alignments_blinker_inter.pickle")
#torch.save(blinker_verse_alignments_gdfa, "/mounts/work/ayyoob/models/gnn/pruned_alignments_blinker_gdfa.pickle")
utils.LOG.info('done saving pruned alignments')

print('reading inter verse alignments')
blinker_verse_alignments_inter = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_blinker_inter.pickle")
#blinker_verse_alignments_gdfa = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_blinker_gdfa.pickle")
gc.collect()
print('done reading inter verse alignments')

verses_map = {}

for edit in blinker_test_dataset.nodes_map:
    for verse in blinker_test_dataset.nodes_map[edit]:
        if verse not in verses_map:
            for tok in blinker_test_dataset.nodes_map[edit][verse]:
                verses_map[verse] = blinker_test_dataset.nodes_map[edit][verse][tok]
                break

sorted_verses = sorted(verses_map.items(), key = lambda x: x[1])
blinker_verses = [item[0] for item in sorted_verses]


2021-10-18 17:59:44,617 - analytics - INFO - done reading alignments
2021-10-18 17:59:44,619 - analytics - INFO - done saving pruned alignments


reading inter verse alignments
done reading inter verse alignments


In [14]:
##importlib.reload(afeatures)
#grc_test_dataset = torch.load("/mounts/work/ayyoob/models/gnn/dataset_helfi_grc_test_community_word.pickle", map_location=torch.device('cpu'))
#editf_fin = "fin-x-bible-helfi"
#editf_grc = 'grc-x-bible-helfi'

#test_gold_grc = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi-fin-grc-gold-alignments_test.txt"

#pros_grc, surs_grc = autils.load_gold(test_gold_grc)
#grc_verses = list(pros_grc.keys())


#grc_test_verse_alignments_inter = {}
#grc_test_verse_alignments_gdfa = {}
#gc.collect()
##args = []
##for i,verse in enumerate(grc_verses):
##    args.append((verse, current_editions))

##with Pool(20) as p:
##    all_res = p.map(get_pruned_verse_alignments, args)

##for i,verse in enumerate(grc_verses):
##    verse_aligns_inter, verse_aligns_gdfa = all_res[i]
    
##    grc_test_verse_alignments_inter[verse] = verse_aligns_inter
##    grc_test_verse_alignments_gdfa[verse] = verse_aligns_gdfa

#utils.LOG.info("done reading alignments")
##torch.save(grc_test_verse_alignments_inter, "/mounts/work/ayyoob/models/gnn/pruned_alignments_grc_inter.pickle")
##torch.save(grc_test_verse_alignments_gdfa, "/mounts/work/ayyoob/models/gnn/pruned_alignments_grc_gdfa.pickle")
#utils.LOG.info('done saving pruned alignments')

#print('reading inter verse alignments')
#grc_test_verse_alignments_inter = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_grc_inter.pickle")
#grc_test_verse_alignments_gdfa = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_grc_gdfa.pickle")
#gc.collect()
#print('done reading inter verse alignments')

#verses_map = {}

#for edit in grc_test_dataset.nodes_map:
#    for verse in grc_test_dataset.nodes_map[edit]:
#        if verse not in verses_map:
#            for tok in grc_test_dataset.nodes_map[edit][verse]:
#                verses_map[verse] = grc_test_dataset.nodes_map[edit][verse][tok]
#                break

#sorted_verses = sorted(verses_map.items(), key = lambda x: x[1])
#grc_test_verses = [item[0] for item in sorted_verses]

#gc.collect()

In [15]:
#heb_test_dataset = torch.load("/mounts/work/ayyoob/models/gnn/dataset_helfi_heb_test_community_word.pickle", map_location=torch.device('cpu'))
#editf_fin = "fin-x-bible-helfi"
#editf_heb = 'heb-x-bible-helfi'

#test_gold_heb = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi-fin-heb-gold-alignments_test.txt"

#pros_heb, surs_heb = autils.load_gold(test_gold_heb)
#heb_verses = list(pros_heb.keys())


#heb_test_verse_alignments_inter = {}
#heb_test_verse_alignments_gdfa = {}
##args = []
##for i,verse in enumerate(heb_verses):
##    args.append((verse, current_editions))

##with Pool(20) as p:
##    all_res = p.map(get_pruned_verse_alignments, args)

##for i,verse in enumerate(heb_verses):
##    verse_aligns_inter, verse_aligns_gdfa = all_res[i]
    
##    heb_test_verse_alignments_inter[verse] = verse_aligns_inter
##    heb_test_verse_alignments_gdfa[verse] = verse_aligns_gdfa

##utils.LOG.info("done reading alignments")
##torch.save(heb_test_verse_alignments_inter, "/mounts/work/ayyoob/models/gnn/pruned_alignments_heb_inter.pickle")
##torch.save(heb_test_verse_alignments_gdfa, "/mounts/work/ayyoob/models/gnn/pruned_alignments_heb_gdfa.pickle")
##utils.LOG.info('done saving pruned alignments')

#print('reading inter verse alignments')
#heb_test_verse_alignments_inter = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_heb_inter.pickle")
#heb_test_verse_alignments_gdfa = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_heb_gdfa.pickle")
#gc.collect()
#print('done reading inter verse alignments')

#verses_map = {}

#for edit in heb_test_dataset.nodes_map:
#    for verse in heb_test_dataset.nodes_map[edit]:
#        if verse not in verses_map:
#            for tok in heb_test_dataset.nodes_map[edit][verse]:
#                verses_map[verse] = heb_test_dataset.nodes_map[edit][verse][tok]
#                break

#sorted_verses = sorted(verses_map.items(), key = lambda x: x[1])
#heb_test_verses = [item[0] for item in sorted_verses]
#gc.collect()

In [16]:
verses_map = {}

for edit in train_dataset.nodes_map:
    for verse in train_dataset.nodes_map[edit]:
        if verse not in verses_map:
            for tok in train_dataset.nodes_map[edit][verse]:
                verses_map[verse] = train_dataset.nodes_map[edit][verse][tok]
                break

sorted_verses = sorted(verses_map.items(), key = lambda x: x[1])
all_verses = [item[0] for item in sorted_verses]

long_verses = set()

for edit in train_dataset.nodes_map.keys():
    for verse in train_dataset.nodes_map[edit]:
        to_print = False
        for tok in train_dataset.nodes_map[edit][verse]:
            if tok > 150:
                to_print = True
        if to_print == True:
            long_verses.add(verse)


train_verses = all_verses[:]

masked_verses = list(long_verses)
masked_verses.extend(blinker_verses)

In [21]:
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import random

data_dir_train = "/mounts/data/proj/ayyoob/align_induction/dataset/dataset_helfi_train_community_word"
data_dir_blinker = "/mounts/data/proj/ayyoob/align_induction/dataset/pruned_alignments_blinker_inter/"

class GNNDatasetPOSTAG(Dataset):

    def __init__(self, dataset, verses, edit_files, alignments, node_cover, pos_labels, data_dir, create_data=False, group_size = 20):
        self.verses = list(verses)
        self.edit_files = list(edit_files)
        self.pos_labels = pos_labels
        self.data_dir = data_dir

        self.items = self.calculate_size( self.verses, group_size, node_cover)
        self.alignments = alignments
        if create_data:
            self.calculate_verse_stats(verses, edit_files, alignments, dataset, data_dir)
    
    def calculate_size(self, verses, group_size, covered_verses):
        res = []
        for verse in verses:
            covered_nodes = covered_verses[verse]
            random.shuffle(covered_nodes)
            items = [covered_nodes[i:i + group_size] for i in range(0, len(covered_nodes), group_size)]
            res.extend([(verse, i) for i in items])

        return res

    def calculate_verse_stats(self,verses, edition_files, alignments, dataset, data_dir):

        min_edge = 0
        for verse in tqdm(verses):
            min_nodes = 99999999999999
            max_nodes = 0
            edges_tmp = [[],[]]
            x_tmp = []
            features = []
            for i,editf1 in enumerate(edition_files):
                for j,editf2 in enumerate(edition_files[i+1:]):
                    aligns = autils.get_aligns(editf1, editf2, alignments[verse])
                    if aligns != None:
                        for align in aligns:
                            try:
                                n1,_ = gutils.node_nom(verse, editf1, align[0], None, dataset.nodes_map, x_tmp, edition_files, features)
                                n2,_ = gutils.node_nom(verse, editf2, align[1], None, dataset.nodes_map, x_tmp, edition_files, features)
                                edges_tmp[0].extend([n1, n2])

                                max_nodes = max(n1, n2, max_nodes)
                                min_nodes = min(n1, n2, min_nodes)
                            except Exception as e:
                                print(editf1, editf2, verse)
                                raise(e)

            self.verse_info = {}

            self.verse_info['padding'] = min_nodes
            
            self.verse_info['x'] = torch.clone(dataset.x[min_nodes:max_nodes+1,:])
            
            self.verse_info['edge_index'] = torch.clone(dataset.edge_index[:, min_edge : min_edge + len(edges_tmp[0])] - min_nodes)

            if torch.min(self.verse_info['edge_index']) != 0:
                print(verse, min_nodes, max_nodes, min_edge, len(edges_tmp[0]))
                print(torch.min(self.verse_info['edge_index']))
            
            if self.verse_info['x'].shape[0] != torch.max(self.verse_info['edge_index']) + 1 :
                print(verse, min_nodes, max_nodes, min_edge, len(edges_tmp[0]))
                print(torch.min(self.verse_info['edge_index']))
            
            min_edge = min_edge + len(edges_tmp[0])

            torch.save(self.verse_info, f"{data_dir}/verses/{verse}_info.torch.bin")
        
        dataset.x = None
        dataset.edge_index = None
        torch.save(dataset, f"{data_dir}/train_dataset_nox_noedge.torch.bin")

            
    
    def __len__(self):
        return len(self.items)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        verse, nodes = self.items[idx]

        self.verse_info = {verse: torch.load(f'{self.data_dir}/verses/{verse}_info.torch.bin')}
         
        return {'nodes':torch.LongTensor(nodes) , 'verse':verse, 'self': self}

def collate_fun(input):
    res = {}
    for item in input:
        verse = item['verse'] 
        if verse not in res:
            res[verse] = {'x':item['self'].verse_info[verse]['x'], 'edge_index':item['self'].verse_info[verse]['edge_index']
                 ,'nodes':torch.LongTensor([]), 'padding':item['self'].verse_info[verse]['padding']}

        res[verse]['nodes'] = torch.cat((item['nodes'], res[verse]['nodes']), dim=0)
    
    for verse in res:
        res[verse]['labels'] = input[0]['self'].pos_labels[ res[verse]['nodes'] ,:]
        res[verse]['nodes'] -=  res[verse]['padding']

     
    return res

#train_pos_labels, train_pos_node_cover = get_pos_tags(train_dataset)
#gnn_dataset_train_pos = GNNDatasetPOSTAG(train_dataset, train_verses, current_editions, verse_alignments_inter,
#                        train_pos_node_cover, train_pos_labels, data_dir_train)
torch.save({'pos_labels':train_pos_labels, 'pos_node_cover':train_pos_node_cover}, f'{data_dir_train}/pos_data.torch.bin')

blinker_pos_labels, blinker_pos_node_cover = get_pos_tags(blinker_test_dataset)
gnn_dataset_blinker_pos = GNNDatasetPOSTAG(blinker_test_dataset, blinker_verses, current_editions, blinker_verse_alignments_inter,
                             blinker_pos_node_cover, blinker_pos_labels, data_dir_blinker)
torch.save({'pos_labels':blinker_pos_labels, 'pos_node_cover': blinker_pos_node_cover}, f'{data_dir_blinker}/pos_data.torch.bin')

len(gnn_dataset_train_pos)
gc.collect()

100%|██████████| 250/250 [00:19<00:00, 12.79it/s]


479366

In [ ]:
from tqdm import tqdm
gnn_dataset = gnn_dataset_blinker_pos
data_loader = DataLoader(gnn_dataset_blinker_pos, batch_size=1, collate_fn=collate_fun, shuffle=True)

clean_memory()
n_head = 1

def turn_off_embedding_updates(encoder):
    for i,ft in enumerate(encoder.feature_types):
        if ft.type == MAPPING:
            encoder.layers[i] = MappingEncoding(encoder.layers[i].emb.weight, freeze=True)


batch_size = 100
channels = 512

in_dim = sum(t.out_dim for t in features)

classifier = Classifier(n_head * channels, len(postag_map))
model.decoder = classifier

print("sending input to gpu")
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S-") + f"samett-{channels}chs-feat{train_dataset.num_node_features}-")

torch.set_printoptions(edgeitems=5)
print("model params - decoder params - conv1", sum(p.numel() for p in model.parameters()), sum(p.numel() for p in decoder.parameters()))

for epoch in range(1, 2):
    print(f"\n----------------epoch {epoch} ---------------")
    
    #if epoch % 1 == 0:
    #    train_neg_edge_index = gutils.get_negative_edges(train_verses, small_editions, train_dataset.nodes_map,  verse_alignments_inter).to(dev)
        #edge_index_seq_sent_neg = get_negative_edges_seq(train_dataset.nodes_map).to(dev)

    train(epoch)
    save_model(model)
    clean_memory()
    if epoch % 1 == 0:
        #alignment_test(epoch, test_dataset.edge_index, editf1, editf2, test_verses[:30], test_nodes_map,
        #    dev, model, x_test, pros, surs, verse_alignments_inter, verse_alignments_gdfa, writer, gnn_dataset.verse_info)
        #eval_utils.alignment_test(epoch, test_dataset.edge_index, editf1, editf2, test_verses[:], test_nodes_map,
        #    dev, model, x_test, pros, surs, verse_alignments_inter, verse_alignments_gdfa, writer, gnn_dataset.verse_info)

        eval_utils.alignment_test(epoch, grc_test_dataset.edge_index, editf_fin, editf_grc, grc_test_verses[:], grc_test_dataset.nodes_map,
                        dev, model, grc_test_dataset.x, pros_grc, surs_grc, grc_test_verse_alignments_inter, grc_test_verse_alignments_gdfa, writer, gnn_dataset_grc.verse_info)

        eval_utils.alignment_test(epoch, heb_test_dataset.edge_index, editf_fin, editf_heb, heb_test_verses[:], heb_test_dataset.nodes_map,
                                dev, model, heb_test_dataset.x, pros_heb, surs_heb, heb_test_verse_alignments_inter, heb_test_verse_alignments_gdfa, writer, gnn_dataset_heb.verse_info)

        eval_utils.alignment_test(epoch, blinker_test_dataset.edge_index, editf12, editf22, blinker_verses, blinker_test_dataset.nodes_map,
                    dev, model, blinker_test_dataset.x, pros_blinker, surs_blinker, blinker_verse_alignments_inter, blinker_verse_alignments_gdfa, writer, gnn_dataset_blinker.verse_info)
        # auc, ap = test(edge_index_seq_sent, edge_index_seq_sent_neg, epoch)
        # print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    clean_memory()

In [ ]:
from gnn_utils import eval_utils
train_dataset.train_mask = train_dataset.val_mask = train_dataset.test_mask = train_dataset.y = None
test_dataset.train_mask = test_dataset.val_mask = test_dataset.test_mask = test_dataset.y = None

torch.cuda.set_device(0)

features = train_dataset.features

device:  cuda


In [ ]:
def save_model(model):
    model.encoder.feature_encoder.feature_types[0] = afeatures.OneHotFeature(20, 83, 'editf')
    model.encoder.feature_encoder.feature_types[1] = afeatures.OneHotFeature(32, 150, 'position')
    model.encoder.feature_encoder.feature_types[2] = afeatures.FloatFeature(4, 'degree_centrality')
    model.encoder.feature_encoder.feature_types[3] = afeatures.FloatFeature(4, 'closeness_centrality')
    model.encoder.feature_encoder.feature_types[4] = afeatures.FloatFeature(4, 'betweenness_centrality')
    model.encoder.feature_encoder.feature_types[5] = afeatures.FloatFeature(4, 'load_centrality')
    model.encoder.feature_encoder.feature_types[6] = afeatures.FloatFeature(4, 'harmonic_centrality')
    model.encoder.feature_encoder.feature_types[7] = afeatures.OneHotFeature(32, 250, 'greedy_modularity_community')
    model.encoder.feature_encoder.feature_types[8] = afeatures.OneHotFeature(32, 250, 'community_2')
    model.encoder.feature_encoder.feature_types[9] = afeatures.MappingFeature(100, 'word')
    torch.save(model, '/mounts/work/ayyoob/models/gnn/checkpoint/gnn_512_flggll_word_halfTrain_nofeatlinear_encoderlineear_decoderonelayer' + datetime.now().strftime("%Y%m%d-%H%M%S-") + '.pickle')

In [ ]:
#from tqdm import tqdm

#torch.cuda.set_device(4)
#features = train_dataset.features

#data_loader = DataLoader(pos_gnn_dataset_train, batch_size=1, shuffle=True)
#test_data_loader = DataLoader(pos_gnn_dataset_dev, batch_size=1, shuffle=True)

#clean_memory()
#drop_out = 0
#n_head = 1

#channels = 512

#decoder_in_dim = n_head * channels 
#decoder = POSDecoder(decoder_in_dim, decoder_in_dim*2, len(postag_map))
#model = torch.load('/mounts/work/ayyoob/models/gnn/checkpoint/gnn_512_flggll_word_halfTrain_nofeatlinear_encoderlineear_decoderonelayer20210910-235352-.pickle')
#model.decoder = decoder
#model.to(dev)
#criterion = nn.CrossEntropyLoss()

#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

#torch.set_printoptions(edgeitems=5)
#print("model params - decoder params - conv1", sum(p.numel() for p in model.parameters()), sum(p.numel() for p in decoder.parameters()))

#for epoch in range(1, 10):
#    print(f"\n----------------epoch {epoch} ---------------")
    
#    #if epoch % 1 == 0:
#    #    train_neg_edge_index = gutils.get_negative_edges(train_verses, small_editions, train_dataset.nodes_map,  verse_alignments_inter).to(dev)
#        #edge_index_seq_sent_neg = get_negative_edges_seq(train_dataset.nodes_map).to(dev)

#    train(epoch)
#    save_model(model)
#    test(epoch, test_data_loader) 
#    clean_memory()

edge features size:  0


  0%|          | 0/109374 [00:00<?, ?it/s]

sending input to gpu
model params - decoder params - conv1 237583457 1050625

----------------epoch 1 ---------------


100%|██████████| 783/783 [00:34<00:00, 22.98it/s]




intersection prec: 0.897, rec: 0.506, F1: 0.647, AER: 0.353
gdfa prec: 0.733, rec: 0.671, F1: 0.701, AER: 0.299
my_gdfa prec: 0.632, rec: 0.78, F1: 0.698, AER: 0.302
argmax prec: 0.845, rec: 0.698, F1: 0.764, AER: 0.235
resnorm prec: 0.549, rec: 0.785, F1: 0.646, AER: 0.354
itermax2-.9 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.95 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.8 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
my_gd prec: 0.815, rec: 0.72, F1: 0.765, AER: 0.235
my_gd_gdfa prec: 0.719, rec: 0.785, F1: 0.751, AER: 0.25
new1 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd_gdfa prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0


...



intersection prec: 0.897, rec: 0.506, F1: 0.647, AER: 0.353
gdfa prec: 0.733, rec: 0.671, F1: 0.701, AER: 0.299
my_gdfa prec: 0.634, rec: 0.783, F1: 0.701, AER: 0.299
argmax prec: 0.847, rec: 0.691, F1: 0.761, AER: 0.239
resnorm prec: 0.632, rec: 0.776, F1: 0.697, AER: 0.303
itermax2-.9 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.95 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.8 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
my_gd prec: 0.822, rec: 0.727, F1: 0.772, AER: 0.228
my_gd_gdfa prec: 0.725, rec: 0.783, F1: 0.753, AER: 0.247
new1 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd_gdfa prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0


  1%|          | 3/250 [00:00<00:11, 21.90it/s]



intersection prec: 0.818, rec: 0.268, F1: 0.404, AER: 0.596
gdfa prec: 0.508, rec: 0.448, F1: 0.476, AER: 0.524
my_gdfa prec: 0.512, rec: 0.744, F1: 0.607, AER: 0.394
argmax prec: 0.776, rec: 0.611, F1: 0.684, AER: 0.316
resnorm prec: 0.574, rec: 0.729, F1: 0.642, AER: 0.358
itermax2-.9 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.95 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.8 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
my_gd prec: 0.765, rec: 0.654, F1: 0.705, AER: 0.295
my_gd_gdfa prec: 0.609, rec: 0.692, F1: 0.648, AER: 0.352
new1 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd_gdfa prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0


100%|██████████| 250/250 [00:13<00:00, 17.97it/s]




intersection prec: 0.971, rec: 0.521, F1: 0.678, AER: 0.319
gdfa prec: 0.856, rec: 0.71, F1: 0.776, AER: 0.221
my_gdfa prec: 0.783, rec: 0.755, F1: 0.769, AER: 0.231
argmax prec: 0.942, rec: 0.673, F1: 0.785, AER: 0.212
resnorm prec: 0.712, rec: 0.76, F1: 0.735, AER: 0.266
itermax2-.9 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.95 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
itermax2-.8 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
my_gd prec: 0.927, rec: 0.708, F1: 0.803, AER: 0.194
my_gd_gdfa prec: 0.867, rec: 0.785, F1: 0.824, AER: 0.174
new1 prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0
new_mygd_gdfa prec: 0.0, rec: 0.0, F1: 0.0, AER: 1.0


In [ ]:
i = sag
batch = khar
verse = gav
print(i, verse)

keys = list(gnn_dataset.verse_info.keys())

gnn_dataset.verse_info[verse]


In [ ]:
data_loader_blinker = DataLoader(gnn_dataset_blinker, batch_size=1, collate_fn=collate_fun, shuffle=True)
data_loader_heb = DataLoader(gnn_dataset_heb, batch_size=1, collate_fn=collate_fun, shuffle=True)
data_loader_grc = DataLoader(gnn_dataset_grc, batch_size=1, collate_fn=collate_fun, shuffle=True)

clean_memory()
data_loader = data_loader_blinker
gnn_dataset = gnn_dataset_blinker
train(1)
clean_memory()
eval_utils.alignment_test(epoch, blinker_test_dataset.edge_index, editf12, editf22, blinker_verses[:], blinker_test_dataset.nodes_map,
                    dev, model, blinker_test_dataset.x, pros_blinker, surs_blinker, blinker_verse_alignments_inter, blinker_verse_alignments_gdfa, writer, gnn_dataset_blinker.verse_info)
clean_memory()

data_loader = data_loader_grc
gnn_dataset = gnn_dataset_grc
train(1)
clean_memory()
eval_utils.alignment_test(epoch, grc_test_dataset.edge_index, editf_fin, editf_grc, grc_test_verses[:], grc_test_dataset.nodes_map,
                        dev, model, grc_test_dataset.x, pros_grc, surs_grc, grc_test_verse_alignments_inter, grc_test_verse_alignments_gdfa, writer, gnn_dataset_grc.verse_info)
clean_memory()

data_loader = data_loader_heb
gnn_dataset = gnn_dataset_heb
train(1)
clean_memory()
eval_utils.alignment_test(epoch, heb_test_dataset.edge_index, editf_fin, editf_heb, heb_test_verses[:], heb_test_dataset.nodes_map,
                        dev, model, heb_test_dataset.x, pros_heb, surs_heb, heb_test_verse_alignments_inter, heb_test_verse_alignments_gdfa, writer, gnn_dataset_heb.verse_info)
clean_memory()


100%|██████████| 1180/1180 [00:32<00:00, 36.58it/s]


train loss: 0
cluster loss: 0


intersection prec: 0.971, rec: 0.521, F1: 0.678, AER: 0.319
gdfa prec: 0.856, rec: 0.71, F1: 0.776, AER: 0.221
my_gdfa prec: 0.783, rec: 0.754, F1: 0.768, AER: 0.231
argmax prec: 0.94, rec: 0.673, F1: 0.784, AER: 0.213
resnorm prec: 0.707, rec: 0.759, F1: 0.732, AER: 0.269
itermax2-.9 prec: 0.852, rec: 0.724, F1: 0.783, AER: 0.214
itermax2-.95 prec: 0.85, rec: 0.724, F1: 0.782, AER: 0.215
itermax2-.8 prec: 0.854, rec: 0.725, F1: 0.784, AER: 0.214
my_gd prec: 0.924, rec: 0.704, F1: 0.799, AER: 0.198
my_gd_gdfa prec: 0.865, rec: 0.784, F1: 0.823, AER: 0.175
new1 prec: 0.945, rec: 0.661, F1: 0.778, AER: 0.219
new_mygd prec: 0.92, rec: 0.706, F1: 0.799, AER: 0.198
new_mygd_gdfa prec: 0.863, rec: 0.786, F1: 0.823, AER: 0.176


 43%|████▎     | 2999/7000 [07:08<09:53,  6.74it/s]



intersection prec: 0.897, rec: 0.506, F1: 0.647, AER: 0.353
gdfa prec: 0.733, rec: 0.671, F1: 0.701, AER: 0.299
my_gdfa prec: 0.647, rec: 0.797, F1: 0.714, AER: 0.286
argmax prec: 0.857, rec: 0.697, F1: 0.769, AER: 0.231
resnorm prec: 0.679, rec: 0.779, F1: 0.726, AER: 0.275
itermax2-.9 prec: 0.736, rec: 0.766, F1: 0.751, AER: 0.25
itermax2-.95 prec: 0.734, rec: 0.764, F1: 0.749, AER: 0.251
itermax2-.8 prec: 0.738, rec: 0.766, F1: 0.752, AER: 0.248
my_gd prec: 0.838, rec: 0.731, F1: 0.781, AER: 0.22
my_gd_gdfa prec: 0.736, rec: 0.786, F1: 0.76, AER: 0.24
new1 prec: 0.874, rec: 0.69, F1: 0.771, AER: 0.229
new_mygd prec: 0.835, rec: 0.732, F1: 0.78, AER: 0.22
new_mygd_gdfa prec: 0.734, rec: 0.786, F1: 0.759, AER: 0.241


intersection prec: 0.818, rec: 0.268, F1: 0.404, AER: 0.596
gdfa prec: 0.508, rec: 0.448, F1: 0.476, AER: 0.524
my_gdfa prec: 0.515, rec: 0.75, F1: 0.611, AER: 0.389
argmax prec: 0.774, rec: 0.607, F1: 0.68, AER: 0.32
resnorm prec: 0.548, rec: 0.727, F1: 0.625, AER: 0.

 43%|████▎     | 3001/7000 [12:08<69:33:39, 62.62s/it]

...



intersection prec: 0.897, rec: 0.506, F1: 0.647, AER: 0.353
gdfa prec: 0.733, rec: 0.671, F1: 0.701, AER: 0.299
my_gdfa prec: 0.643, rec: 0.79, F1: 0.709, AER: 0.291
argmax prec: 0.852, rec: 0.7, F1: 0.769, AER: 0.231
resnorm prec: 0.66, rec: 0.778, F1: 0.714, AER: 0.286
itermax2-.9 prec: 0.738, rec: 0.76, F1: 0.749, AER: 0.251
itermax2-.95 prec: 0.737, rec: 0.76, F1: 0.748, AER: 0.252
itermax2-.8 prec: 0.739, rec: 0.76, F1: 0.749, AER: 0.251
my_gd prec: 0.834, rec: 0.728, F1: 0.777, AER: 0.222
my_gd_gdfa prec: 0.734, rec: 0.784, F1: 0.758, AER: 0.242
new1 prec: 0.871, rec: 0.692, F1: 0.771, AER: 0.229
new_mygd prec: 0.832, rec: 0.729, F1: 0.777, AER: 0.223
new_mygd_gdfa prec: 0.732, rec: 0.785, F1: 0.758, AER: 0.242


intersection prec: 0.818, rec: 0.268, F1: 0.404, AER: 0.596
gdfa prec: 0.508, rec: 0.448, F1: 0.476, AER: 0.524
my_gdfa prec: 0.516, rec: 0.751, F1: 0.612, AER: 0.388
argmax prec: 0.78, rec: 0.612, F1: 0.686, AER: 0.314
resnorm prec: 0.568, rec: 0.728, F1: 0.638, AER: 

 90%|█████████ | 5998/6635 [15:47<01:30,  7.04it/s]



intersection prec: 0.897, rec: 0.506, F1: 0.647, AER: 0.353
gdfa prec: 0.733, rec: 0.671, F1: 0.701, AER: 0.299
my_gdfa prec: 0.646, rec: 0.796, F1: 0.713, AER: 0.287
argmax prec: 0.858, rec: 0.699, F1: 0.77, AER: 0.229
resnorm prec: 0.609, rec: 0.791, F1: 0.688, AER: 0.312
itermax2-.9 prec: 0.745, rec: 0.764, F1: 0.754, AER: 0.246
itermax2-.95 prec: 0.743, rec: 0.763, F1: 0.753, AER: 0.247
itermax2-.8 prec: 0.746, rec: 0.764, F1: 0.755, AER: 0.245
my_gd prec: 0.831, rec: 0.738, F1: 0.782, AER: 0.218
my_gd_gdfa prec: 0.733, rec: 0.789, F1: 0.76, AER: 0.24
new1 prec: 0.874, rec: 0.696, F1: 0.775, AER: 0.225
new_mygd prec: 0.829, rec: 0.741, F1: 0.783, AER: 0.218
new_mygd_gdfa prec: 0.731, rec: 0.791, F1: 0.76, AER: 0.24


intersection prec: 0.818, rec: 0.268, F1: 0.404, AER: 0.596
gdfa prec: 0.508, rec: 0.448, F1: 0.476, AER: 0.524
my_gdfa prec: 0.512, rec: 0.749, F1: 0.608, AER: 0.392
argmax prec: 0.783, rec: 0.608, F1: 0.684, AER: 0.316
resnorm prec: 0.529, rec: 0.74, F1: 0.617, AER

100%|██████████| 6635/6635 [21:55<00:00,  5.04it/s]


train loss: 3504449.5449852943
cluster loss: 509.94970703125


intersection prec: 0.818, rec: 0.268, F1: 0.404, AER: 0.596
gdfa prec: 0.508, rec: 0.448, F1: 0.476, AER: 0.524
my_gdfa prec: 0.516, rec: 0.752, F1: 0.612, AER: 0.388
argmax prec: 0.784, rec: 0.612, F1: 0.687, AER: 0.313
resnorm prec: 0.533, rec: 0.744, F1: 0.621, AER: 0.379
itermax2-.9 prec: 0.655, rec: 0.7, F1: 0.677, AER: 0.324
itermax2-.95 prec: 0.652, rec: 0.698, F1: 0.674, AER: 0.325
itermax2-.8 prec: 0.657, rec: 0.7, F1: 0.678, AER: 0.322
my_gd prec: 0.757, rec: 0.665, F1: 0.708, AER: 0.292
my_gd_gdfa prec: 0.607, rec: 0.699, F1: 0.65, AER: 0.35
new1 prec: 0.809, rec: 0.617, F1: 0.7, AER: 0.3
new_mygd prec: 0.751, rec: 0.669, F1: 0.708, AER: 0.293
new_mygd_gdfa prec: 0.604, rec: 0.703, F1: 0.65, AER: 0.351


In [ ]:
from gnn_utils import eval_utils
importlib.reload(eval_utils)

clean_memory()
eval_utils.alignment_test(epoch, heb_test_dataset.edge_index, editf_fin, editf_heb, heb_test_verses, heb_test_dataset.nodes_map,
                        dev, model, heb_test_dataset.x, pros_heb, surs_heb, heb_test_verse_alignments_inter, heb_test_verse_alignments_gdfa, writer, gnn_dataset_heb.verse_info)

eval_utils.alignment_test(epoch, grc_test_dataset.edge_index, editf_fin, editf_grc, grc_test_verses, grc_test_dataset.nodes_map,
                        dev, model, grc_test_dataset.x, pros_grc, surs_grc, grc_test_verse_alignments_inter, grc_test_verse_alignments_gdfa, writer, gnn_dataset_grc.verse_info)

eval_utils.alignment_test(epoch, blinker_test_dataset.edge_index, editf12, editf22, blinker_verses, blinker_test_dataset.nodes_map,
                    dev, model, blinker_test_dataset.x, pros_blinker, surs_blinker, blinker_verse_alignments_inter, blinker_verse_alignments_gdfa, writer, gnn_dataset_blinker.verse_info)

clean_memory()




intersection prec: 0.818, rec: 0.268, F1: 0.404, AER: 0.596
gdfa prec: 0.508, rec: 0.448, F1: 0.476, AER: 0.524
my_gdfa prec: 0.521, rec: 0.757, F1: 0.617, AER: 0.383
argmax prec: 0.781, rec: 0.618, F1: 0.69, AER: 0.31
resnorm prec: 0.513, rec: 0.758, F1: 0.612, AER: 0.388
itermax2-.9 prec: 0.657, rec: 0.702, F1: 0.679, AER: 0.321
itermax2-.95 prec: 0.655, rec: 0.701, F1: 0.677, AER: 0.323
itermax2-.8 prec: 0.66, rec: 0.702, F1: 0.68, AER: 0.32
my_gd prec: 0.759, rec: 0.663, F1: 0.708, AER: 0.292
my_gd_gdfa prec: 0.607, rec: 0.701, F1: 0.651, AER: 0.349
new1 prec: 0.808, rec: 0.616, F1: 0.699, AER: 0.3
new_mygd prec: 0.753, rec: 0.667, F1: 0.707, AER: 0.293
new_mygd_gdfa prec: 0.604, rec: 0.704, F1: 0.65, AER: 0.35


intersection prec: 0.897, rec: 0.506, F1: 0.647, AER: 0.353
gdfa prec: 0.733, rec: 0.671, F1: 0.701, AER: 0.299
my_gdfa prec: 0.643, rec: 0.792, F1: 0.71, AER: 0.29
argmax prec: 0.85, rec: 0.7, F1: 0.768, AER: 0.233
resnorm prec: 0.582, rec: 0.791, F1: 0.671, AER: 0.329


In [ ]:
# produce for uruba
importlib.reload(eval_utils)
editf_yor = 'yor-x-bible-2010'
editf_others = ['eng-x-bible-mixed', 'deu-x-bible-newworld', 'ces-x-bible-newworld', 'fra-x-bible-louissegond', 'hin-x-bible-newworld',
                'ita-x-bible-2009', 'prs-x-bible-goodnews', 'ron-x-bible-2006', 'spa-x-bible-newworld']

#def get_pruned_verse_alignments(args):
#    verse, current_editions = args
    
#    #verse_aligns_inter = autils.get_verse_alignments(verse)
#    verse_aligns_gdfa = autils.get_verse_alignments(verse, gdfa=True)

#    #autils.prune_non_necessary_alignments(verse_aligns_inter, current_editions)
#    autils.prune_non_necessary_alignments(verse_aligns_gdfa, current_editions)

#    gc.collect()
#    return verse_aligns_gdfa
    

#verse_alignments_gdfa = {}
#args = []
#editfs = editf_others[:]
#editfs.append(editf_yor)
#for i,verse in enumerate(train_verses):
#    args.append((verse, editfs))

#print('going to get alignments')
#with Pool(20) as p:
#    all_res = p.map(get_pruned_verse_alignments, args)

#for i,verse in enumerate(all_verses):
#    verse_aligns_gdfa = all_res[i]
    
#    verse_alignments_gdfa[verse] = verse_aligns_gdfa

for verse in train_dataset.nodes_map[editf_yor]:
    if verse not in surs :
        surs[verse] = set()
        pros[verse] = set()

#verse_alignments_gdfa = torch.load("/mounts/work/ayyoob/models/gnn/pruned_alignments_train_gdfa_yoruba.pickle")

for eidtf_t in editf_others:
    res = {}

    print('going to align heb ')
    if eidtf_t in heb_test_dataset.nodes_map:
        verses = set(heb_test_dataset.nodes_map[editf_yor].keys()).intersection(heb_test_dataset.nodes_map[eidtf_t].keys())
        res_ = eval_utils.alignment_test(epoch, heb_test_dataset.edge_index, editf_yor, eidtf_t, list(verses), heb_test_dataset.nodes_map,
                        dev, model, heb_test_dataset.x, pros_heb, surs_heb, heb_test_verse_alignments_inter, heb_test_verse_alignments_gdfa,
                        writer, gnn_dataset_heb.verse_info, calc_numbers=False)
        clean_memory()
        res.update(res_)

    print('going to align train ')
    verses = set(train_dataset.nodes_map[editf_yor].keys()).intersection(train_dataset.nodes_map[eidtf_t].keys())
    res_ = eval_utils.alignment_test(epoch, train_dataset.edge_index, editf_yor, eidtf_t, list(verses - set(masked_verses)), train_dataset.nodes_map,
                    dev, model, train_dataset.x, pros, surs, verse_alignments_inter, verse_alignments_gdfa, writer, gnn_dataset_train.verse_info, calc_numbers=False)
    clean_memory()
    res.update(res_)

    print('going to align blinker ')
    verses = set(blinker_test_dataset.nodes_map[editf_yor].keys()).intersection(blinker_test_dataset.nodes_map[eidtf_t].keys())
    res_ = eval_utils.alignment_test(epoch, blinker_test_dataset.edge_index, editf_yor, eidtf_t, list(verses), blinker_test_dataset.nodes_map,
                    dev, model, blinker_test_dataset.x, pros_blinker, surs_blinker, blinker_verse_alignments_inter, blinker_verse_alignments_gdfa,
                    writer, gnn_dataset_blinker.verse_info, calc_numbers=False)
    clean_memory()
    res.update(res_)

    print('going to align grc ')
    verses = set(grc_test_dataset.nodes_map[editf_yor].keys()).intersection(grc_test_dataset.nodes_map[eidtf_t].keys())
    res_ = eval_utils.alignment_test(epoch, grc_test_dataset.edge_index, editf_yor, eidtf_t, list(verses), grc_test_dataset.nodes_map,
                    dev, model, grc_test_dataset.x, pros_grc, surs_grc, grc_test_verse_alignments_inter, grc_test_verse_alignments_gdfa,
                    writer, gnn_dataset_grc.verse_info, calc_numbers=False)
    clean_memory()
    res.update(res_)

    
    
    print(f'going save alignments for {eidtf_t}')
    torch.save(res, f'/mounts/work/ayyoob/results/gnn_align/yoruba/{eidtf_t}_alignments.bin')

  0%|          | 2/6202 [00:00<06:39, 15.54it/s]

going to align heb 
going to align train 


  3%|▎         | 2/73 [00:00<00:04, 17.74it/s]

going to align blinker 


  0%|          | 2/783 [00:00<00:42, 18.52it/s]

...

going to align blinker 


  0%|          | 2/783 [00:00<00:45, 17.01it/s]

going to align grc 


100%|██████████| 783/783 [00:40<00:00, 19.29it/s]


going save alignments for spa-x-bible-newworld


In [ ]:
global model, decoder
#1/0

decoder = None
model = None

gc.collect()
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:

features = blinker_test_dataset.features[:]
#features_edge = train_dataset.features_edge[:]
from pprint import pprint
#print('indim',in_dim)
#features[-1].out_dim = 50
for i in features:
    #if i.type==3:
    #    i.out_dim=4
    print(vars(i))

sum(p.out_dim for p in features)
#train_dataset.features.pop()
#train_dataset.features[0] = afeatures.OneHotFeature(20, 83, 'editf')
#train_dataset.features[1] = afeatures.OneHotFeature(32, 150, 'position')
#train_dataset.features[2] = afeatures.FloatFeature(4, 'degree_centrality')
#train_dataset.features[3] = afeatures.FloatFeature(4, 'closeness_centrality')
#train_dataset.features[4] = afeatures.FloatFeature(4, 'betweenness_centrality')
#train_dataset.features[5] = afeatures.FloatFeature(4, 'load_centrality')
#train_dataset.features[6] = afeatures.FloatFeature(4, 'harmonic_centrality')
#train_dataset.features[7] = afeatures.OneHotFeature(32, 250, 'greedy_modularity_community')
##train_dataset.features.append(afeatures.MappingFeature(100, 'word'))
#torch.save(train_dataset, "/mounts/work/ayyoob/models/gnn/dataset_helfi_train_community_word.pickle")
#torch.save(train_dataset.features[-3], "./features.tmp")

In [ ]:
# count number of deleted edges by each community detection method
# from networkx.algorithms.community import greedy_modularity_communities, asyn_lpa_communities, label_propagation_communities, asyn_fluidc

# tmp_verses = [all_verses[2]]
# tmp_editions = small_editions[:10]
# tmp_dataset, tmp_nodes_map = create_dataset(tmp_verses, verse_alignments_inter, tmp_editions)

# tmp_g = pyg_utils.convert.to_networkx(tmp_dataset, to_undirected=True)
# def count_deleted_edges(tmp_dataset, c):
#     deleted_edges = 0
#     for i in range(0, len(tmp_dataset.edge_index[0]), 2):
#         for comp in c:
#             if tmp_dataset.edge_index[0][i].item() in comp and tmp_dataset.edge_index[1][i].item() not in comp:
#                 deleted_edges += 1
    
#     return deleted_edges

# print("eng token count: ", tmp_nodes_map['eng-x-bible-mixed'][tmp_verses[0]])
# print("original connected components",nx.number_connected_components(tmp_g))

# c = list(greedy_modularity_communities(tmp_g))
# print("new connected_components", len(c))
# print("deleted edges: ", count_deleted_edges(tmp_dataset, c))

# c = list(asyn_lpa_communities(tmp_g))
# print("asyn_lpa_communities number of components", len(c))
# print("deleted edges: ", count_deleted_edges(tmp_dataset, c))

# c = list(label_propagation_communities(tmp_g))
# print("label_propagation_communities number of components", len(c))
# print("deleted edges: ", count_deleted_edges(tmp_dataset, c))

# cents = nx.edge_betweenness_centrality(tmp_g)
# vals = sorted(list(cents.values()))
# print(vals[0], vals[10], vals[100], vals[1000], vals[2000], vals[3000], vals[10000])
# print(vals[-1], vals[-10], vals[-100], vals[-1000], vals[-2000], vals[-3000], vals[-10000])



In [ ]:
# measure different community detection algorithms
# from networkx.algorithms.community import greedy_modularity_communities, asyn_lpa_communities, label_propagation_communities, asyn_fluidc

# def remove_bad_community_edges(nodes_map, verses, edition_files, alignments):
#     edges_tmp = [[],[]]
#     res_edges = [[],[]]
#     for verse in verses:
#         utils.LOG.info(f"extracting edge features for {verse}")
#         for i,editf1 in enumerate(edition_files):
#             for j,editf2 in enumerate(edition_files[i+1:]):
#                 aligns = autils.get_aligns(editf1, editf2, alignments[verse])
#                 if aligns != None:
#                     for align in aligns:
#                         n1, node_count = node_nom(verse, editf1, align[0], 0, nodes_map, None, None)
#                         n2, node_count = node_nom(verse, editf2, align[1], 0, nodes_map, None, None)
#                         edges_tmp[0].extend([n1, n2])
#                         edges_tmp[1].extend([n2, n1])

#         gnx = convert_to_netx(edges_tmp)
#         print('detecting communities')
#         coms = greedy_modularity_communities(gnx)

#         print('finding good edges')
#         for i in range(0, len(edges_tmp[0]), 2):
#             for c in coms:
#                 if edges_tmp[0][i] in c and edges_tmp[0][i+1] in c:
#                     res_edges[0].extend([edges_tmp[0][i], edges_tmp[0][i+1]])
#                     res_edges[1].extend([edges_tmp[0][i+1], edges_tmp[0][i]])
#         edges_tmp = [[],[]]
#     print('to keep edges:', len(res_edges[0]))
#     return torch.tensor(res_edges, dtype=torch.long)

# # old_edge_index = train_dataset.edge_index
# # new_edge_index = remove_bad_community_edges(train_dataset.nodes_map, train_verses, small_editions, verse_alignments_inter)
# # train_dataset.edge_index = new_edge_index

# # with open("./dataset_greedy_modularity_communities.pickle", 'rb') as inf:
# #     train_dataset = pickle.load(inf)

# test_dataset = train_dataset

# print('orig edge count', old_edge_index.shape)
# print('new edge count', train_dataset.edge_index.shape)
# print("done")

In [ ]:
nodes_map = train_dataset.nodes_map
bad_edition_files = []
for edit in nodes_map:
    bad_count = 0
    for verse in nodes_map[edit]:
        if len(nodes_map[edit][verse].keys()) < 2:
            bad_count += 1
        if bad_count > 1:
            bad_edition_files.append(edit)
            break
print(bad_edition_files)

In [ ]:
all_japanese_nodes = set()
nodes_map = train_dataset.nodes_map

for bad_editionf in bad_edition_files:
    for verse in nodes_map[bad_editionf]:
        for item in nodes_map[bad_editionf][verse].items():
            all_japanese_nodes.add(item[1])

print(" all japansese nodes: ", len(all_japanese_nodes))
edge_index = train_dataset.edge_index.to('cpu')
remaining_edges_index = []
for i in tqdm(range(0, edge_index.shape[1], 2)):
    if edge_index[0, i].item() not in all_japanese_nodes and edge_index[0, i+1].item() not in all_japanese_nodes:
        remaining_edges_index.extend([i, i+1])

print('original total edges count', edge_index.shape)
print('remaining edge count', len(remaining_edges_index))
train_dataset.edge_index = edge_index[:, remaining_edges_index]
train_dataset.edge_index.shape


In [ ]:
from networkx.algorithms.community import greedy_modularity_communities, asyn_lpa_communities, label_propagation_communities, asyn_fluidc

def get_community_edges(c, verse):
    res = []
    for n1 in all_nodes_map[editf1][verse].items():
        for n2 in all_nodes_map[editf2][verse].items():
            for com in c:
                if n1[1] in com and n2[1] in com:
                    res.append((n1[0], n2[0]))
    
    return res

def intersect(e1, e2):
    res = set()
    for item in e1:
        if item in e2:
            res.add(item)
    
    return res


measures = {}
measures['intersection']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['gdfa']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c1_all']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c1_inter']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c2_all']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c2_inter']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c3_all']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c3_inter']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
numbers = [100, 300, 600, 1000, 1300, 1600, 2000, 3000, 4000, 5000]
for i in numbers:
    measures[i] = {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
    measures[i+1] = {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
coms = {}
for verse in test_verses:
    inter_edges = autils.get_aligns(editf1, editf2, verse_alignments_inter[verse])

    all_dataset, all_nodes_map = create_dataset([verse], verse_alignments_inter, small_editions)
    print("converting")
    g = pyg_utils.convert.to_networkx(all_dataset, to_undirected=True)
    print("detecting community 1")
    c1 = list(greedy_modularity_communities(g))
    print("detecting community 2")
    c2 = list(asyn_lpa_communities(g))
    print("detecting community 3")
    c3 = list(label_propagation_communities(g))

    print("detecting community 4")
    edge_betweenness = [y[0] for y in sorted(nx.edge_betweenness_centrality(g).items(), key=lambda x: x[1], reverse=True)]
    
    print('orig communities', nx.number_connected_components(g))
    prev_i = 0
    for i in numbers:
        for j in range(prev_i, i):
            g.remove_edge(*edge_betweenness[j])
        prev_i = i
        com = list(nx.connected_components(g))
        edges = get_community_edges(com, verse)
        autils.calc_and_update_alignment_score(edges, pros[verse], surs[verse], measures[i])
        autils.calc_and_update_alignment_score(intersect(edges, inter_edges), pros[verse], surs[verse], measures[i+1]) 
        print(f'communities {i}', nx.number_connected_components(g))

    c1_edges = get_community_edges(c1, verse)
    c2_edges = get_community_edges(c2, verse)
    c3_edges = get_community_edges(c3, verse)
    print('c1 communities', len(c1))
    print('c2 communities', len(c2))
    print('c3 communities', len(c3))



    autils.calc_and_update_alignment_score(inter_edges, pros[verse], surs[verse], measures['intersection'])
    autils.calc_and_update_alignment_score(autils.get_aligns(editf1, editf2, verse_alignments_gdfa[verse]), pros[verse], surs[verse], measures['gdfa'])

    autils.calc_and_update_alignment_score(c1_edges, pros[verse], surs[verse], measures['c1_all'])
    autils.calc_and_update_alignment_score(c2_edges, pros[verse], surs[verse], measures['c2_all'])
    autils.calc_and_update_alignment_score(c3_edges, pros[verse], surs[verse], measures['c3_all'])

    autils.calc_and_update_alignment_score(intersect(c1_edges, inter_edges), pros[verse], surs[verse], measures['c1_inter'])
    autils.calc_and_update_alignment_score(intersect(c2_edges, inter_edges), pros[verse], surs[verse], measures['c2_inter'])
    autils.calc_and_update_alignment_score(intersect(c3_edges, inter_edges), pros[verse], surs[verse], measures['c3_inter'])

    for item in measures:
        print(item, measures[item])
    
    
